In [5]:
%load_ext autoreload
%autoreload 2 

In [6]:
import torch
from gptmodel import GPTModel

GPT_CONFIG_124M = {
    "vocab_size": 50257,
    "context_length": 256,
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,
    "qkv_bias": False
}
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval()

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(256, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=False)
        (W_value): Linear(in_features=768, out_features=768, bias=False)
        (W_key): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features

In [7]:
import tiktoken 
from generate import generate_text_simple

def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0)
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0)
    return tokenizer.decode(flat.tolist())

start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)
print("output text: \n", token_ids_to_text(token_ids, tokenizer))

output text: 
 Every effort moves you Laur inhab plent HoleMen bandwidthillary limits infected Fu


# Using two batches example

In [8]:
inputs = torch.tensor([[16833, 3626, 6100], # every effort moves
                      [40, 1107, 588]]) # I really like
targets = torch.tensor([[3626, 6100, 345], # effort moves you
                       [1107, 588, 11311]]) # really like chocolate

In [9]:
with torch.no_grad():
    logits = model(inputs)
probas = torch.softmax(logits, dim=-1)
print(probas.shape)

torch.Size([2, 3, 50257])


In [10]:
# obtain the resulting token ids
token_ids = torch.argmax(probas, dim=-1, keepdim=True)
print(f"Targets batch 1: {token_ids_to_text(targets[0], tokenizer)}")
print(f"Outputs batch 2: {token_ids_to_text(token_ids[1].flatten(), tokenizer)}")




Targets batch 1:  effort moves you
Outputs batch 2:  SOLD saves Household


In [11]:

# getting the probabilities of the targets in the result
text_idx = 0
target_probas_1 = probas[text_idx, [0,1,2], targets[text_idx]]
print("Text 1 : ", target_probas_1)
text_idx = 1
target_probas_2 = probas[text_idx, [0,1,2], targets[text_idx]]
print("Text 2 : ", target_probas_2)

Text 1 :  tensor([4.1675e-05, 2.2534e-05, 1.1546e-05])
Text 2 :  tensor([1.3597e-05, 4.0070e-05, 4.4748e-06])


## Calculating the loss

In [12]:
log_probas = torch.log(torch.cat((target_probas_1, target_probas_2)))
print(log_probas)
avg_log_probas = torch.mean(log_probas)
print(avg_log_probas)
neg_avg_log_probas = avg_log_probas * -1
print(neg_avg_log_probas)

tensor([-10.0856, -10.7005, -11.3692, -11.2056, -10.1249, -12.3170])
tensor(-10.9671)
tensor(10.9671)


### Applying Cross Entropy Loss Function

In [13]:
# recalling the shape of the logits and target tensors
print("logits shape:", logits.shape)
print("targets shape:", targets.shape)

logits shape: torch.Size([2, 3, 50257])
targets shape: torch.Size([2, 3])


In [14]:
logits_flat = logits.flatten(0, 1) # contains unscaled model outputs before the softmax function
targets_flat = targets.flatten()
print("Flattened logits:", logits_flat.shape)
print("flattened targets:", targets_flat.shape)

Flattened logits: torch.Size([6, 50257])
flattened targets: torch.Size([6])


In [15]:
# the result of this 'loss' is the same as neg_avg_log_probas
loss = torch.nn.functional.cross_entropy(logits_flat, targets_flat)
print(loss)

tensor(10.9671)


In [16]:
file_path = "the-verdict.txt"
with open(file_path, "r", encoding="utf-8") as file:
    text_data = file.read()


In [17]:
total_characters = len(text_data)
total_tokens = len(tokenizer.encode(text_data))
print("Characters: ", total_characters)
print("Tokens: ", total_tokens)

Characters:  20479
Tokens:  5145


In [18]:
train_ratio = 0.9
split_idx = int(train_ratio * total_characters)
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]

print(split_idx)

18431


In [57]:
from dataloader import create_dataloader_v1

torch.manual_seed(123)

train_loader = create_dataloader_v1(
    train_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)
val_loader = create_dataloader_v1(
    val_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)

len 18
len 2


In [58]:
print("Train loader:")
for x,y in train_loader:
    print(x.shape, y.shape)

print("Validation loader:")
for x,y in val_loader:
    print(x.shape, y.shape)



Train loader:
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
Validation loader:
torch.Size([2, 256]) torch.Size([2, 256])


In [59]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch = input_batch.to(device) # this device allows us to transfer the data to a GPU
    target_batch = target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(
        logits.flatten(0, 1), target_batch.flatten()
    )
    return loss



In [65]:
def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(
                input_batch, target_batch, model, device
            )
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches # calculate the average loss over all batches

In [67]:
DEFAULT_DEVICE = "cpu"
if torch.cuda.is_available():
    DEFAULT_DEVICE = "cuda"
elif torch.mps.is_available():
    DEFAULT_DEVICE = "mps"

model = model.to(DEFAULT_DEVICE)
with torch.no_grad():
    train_loss = calc_loss_loader(train_loader, model, DEFAULT_DEVICE)
    val_loss = calc_loss_loader(val_loader, model, DEFAULT_DEVICE)

print("Training loss:", train_loss)
print("Validation loss:", val_loss)
    

Training loss: 10.989001803927952
Validation loss: 10.975936889648438
